In [ ]:
before = pd.read_csv("driving_baseline.csv")
after = pd.read_csv("driving_mod.csv")
compare_out_file = "driving_mod_compare.csv"

In [ ]:
df = pd.merge(before, after, on="Name", how="inner", suffixes=("_before", "_after"))

In [ ]:
df['diff_hh'] = df['weighted_hh_after'] - df['weighted_hh_before']
df['diff_jobs'] = df['weighted_jobs_after'] - df['weighted_jobs_before']
df['diff_ato'] = df['ato_after'] - df['ato_before']

In [ ]:
df.groupby('Origin_CO_TAZID_before')['diff_ato'].sum()

In [ ]:
df.head()

In [ ]:
# assert df['diff_ato'].min() >= 0

In [ ]:
summary = df.groupby('Origin_CO_TAZID_before').agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

In [ ]:
summary.to_csv(compare_out_file)

In [ ]:
df.agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

In [ ]:
arcpy.management.MakeFeatureLayer(
    os.path.join(base_path, r"shp\taz_wfrc.gdb\ATO"), 
    "taz_compare"
)

In [ ]:
arcpy.conversion.TableToTable(
    os.path.join(base_path, compare_out_file), 
    base_gdb , 
    "mod_compare"
)

In [ ]:
arcpy.management.AddJoin(
    "taz_compare", 
    "CO_TAZID", 
    "mod_compare", 
    "Origin_CO_TAZID_before", 
    "KEEP_ALL"
)

In [ ]:
arcpy.management.ApplySymbologyFromLayer("taz_compare", "taz_compare.lyrx")